In [21]:
import pandas as pd

# Liste des mois à traiter
list_mois = ["Mai", "Juin", "Juillet"]

# Chemins vers les fichiers
base_year_file = 'PB2014.xlsx'  # Fichier de prix de base
weight_file = 'indice et pond logement ihpc 2014 zone eco.xlsx'  # Fichier de pondération

# Importer les fichiers Excel
df_base_year = pd.read_excel(base_year_file)
weights_df = pd.read_excel(weight_file)

for mois in list_mois:
    # Chemin vers le fichier du mois en cours
    current_month_file = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Traitements\\Datasets\\{mois.lower()}.csv'  # Fichier de prix du mois en cours

    # Importer le fichier CSV du mois en cours
    df_current_month = pd.read_csv(current_month_file)

    # Calculer la moyenne des prix par catégorie pour l'année de base
    base_year_mean_prices = df_base_year.groupby('code_var 2014')['prix de base'].mean().reset_index()
    base_year_mean_prices.columns = ['code_var 2014', 'prix de base']

    # Calculer la moyenne des prix par catégorie pour le mois en cours
    current_month_mean_prices = df_current_month.groupby('code_var 2014')['price'].mean().reset_index()
    current_month_mean_prices.columns = ['code_var 2014', f'prix_{mois.lower()}_2024']

    # Fusionner les données de l'année de base, du mois en cours, et les pondérations
    merged_df = pd.merge(current_month_mean_prices, base_year_mean_prices, on='code_var 2014')
    merged_df = pd.merge(merged_df, weights_df, on='code_var 2014')

    # Calculer l'indice de Laspeyres par catégorie
    merged_df[f'indice_elementaire_{mois.lower()}_2024'] = (merged_df[f'prix_{mois.lower()}_2024'] / merged_df['prix de base'])

    # Sélectionner uniquement les colonnes pertinentes
    merged_df = merged_df[['code_var 2014', f'prix_{mois.lower()}_2024', 'prix de base', f'indice_elementaire_{mois.lower()}_2024', 'ponderation national']]

    # Calculer l'indice global de Laspeyres
    laspeyres_index_global = (merged_df[f'indice_elementaire_{mois.lower()}_2024'] * merged_df['ponderation national']).sum() / merged_df['ponderation national'].sum() * 100

    # Afficher les résultats
    print(merged_df)
    print(f"Indice de Laspeyres global pour {mois}: {laspeyres_index_global}")

    # Exporter les résultats vers un fichier CSV si nécessaire
    output_path = f'resultats\\laspeyres_index_{mois.lower()}_results.csv'
    merged_df.to_csv(output_path, index=False)
    print(f"Les résultats ont été sauvegardés dans {output_path}")



   code_var 2014  prix_mai_2024   prix de base  indice_elementaire_mai_2024  \
0      401000002   9.398451e+05   90777.800000                    10.353249   
1      401000006   1.900000e+06  216791.600000                     8.764177   
2      401000008   6.866667e+05   75451.333333                     9.100789   
3      401000010   2.433333e+06  233269.230769                    10.431437   

   ponderation national  
0                 75.21  
1                 37.37  
2                 29.15  
3                 53.40  
Indice de Laspeyres global pour Mai: 988.3215693639034
Les résultats ont été sauvegardés dans resultats\laspeyres_index_mai_results.csv
   code_var 2014  prix_juin_2024   prix de base  indice_elementaire_juin_2024  \
0      401000002    8.169816e+05   90777.800000                      8.999795   
1      401000006    1.657692e+06  216791.600000                      7.646478   
2      401000008    3.376667e+05   75451.333333                      4.475291   
3      4010000